In [2]:
import numpy as np
from math import log
import pandas as pd
data = pd.read_csv("tbc.csv")
labels = data.columns
labels = list(labels)
dataset = data.values
print(dataset)
dataset = list(dataset)
print(labels)

[['mittel' ' stark' ' Flocken' ' normal' ' blubbernd'
  ' Lungenentzuendung ']
 ['niedrig' ' leicht' ' Punkt' ' normal' ' fiepend' ' Lungenentzuendung ']
 ['hoch' ' mittel' ' Flocken' ' normal' ' blubbernd'
  ' Lungenentzuendung ']
 ['mittel' ' leicht' ' Flocken' ' normal' ' blubbernd'
  ' Lungenentzuendung ']
 ['ohne' ' leicht' ' Streifen' ' normal' ' normal' ' Tuberkulose ']
 ['hoch' ' stark' ' Loch' ' schnell' ' fiepend' ' Tuberkulose ']
 ['niedrig' ' leicht' ' Streifen' ' normal' ' normal' ' Tuberkulose ']
 ['ohne' ' leicht' ' Punkt' ' schnell' ' fiepend' ' Tuberkulose ']
 ['niedrig' ' mittel' ' Flocken' ' schnell' ' normal' ' Tuberkulose ']]
['hoch', ' stark', ' Flocken', ' normal', ' blubbernd', ' Lungenentzuendung ']


In [3]:
def entropy(dataset):
    m = len(dataset)
    lableCount = {}
    #计数
    for data in dataset:
        currentLabel = data[-1]#最后一行 0或1
        if currentLabel not in lableCount.keys():
            lableCount[currentLabel] = 0
        lableCount[currentLabel] += 1
    
    entropy = 0
    for label in lableCount:
        p = float(lableCount[label]) / m#计算概率
        entropy -= p * log(p,2)#计算熵
    return entropy
 

def split_data(dataset,axis,value):
    """
    划分子数据集
    dataset：数据集
    axis：所选特征的列索引值
    value：所选特征的值
    
    return：子数据集（去掉所选特征那一列之后的数据集
    """
    subSet = []
    for data in dataset:
        if(data[axis] == value):
            data_x = data[:axis]
            data_x = np.hstack((data_x,data[axis+1:]))
#             data_x.hstack(data[axis+1:])
            subSet.append(data_x)
    return subSet
 

def chooseBestFeature(dataSet):
    """
    用信息增益为判断标准 选择最好的特征
    dataset：数据集
    
    best_feature:选出的最好特征 是一个数字
    """
    feature_num = len(dataSet[0])-1
    origin_entropy = entropy(dataSet)
    infoGain = 0
    best_infogain = 0
    for i in range(feature_num):
        value = [data[i] for data in dataSet]
        value_set = set(value)#去重
        #print fi_all
        subset_entropy = 0
        '''遍历所有可能value'''
        for value in value_set:#value指特征的值
            #划分子集
            #print i,value
            subset = split_data(dataSet,i,value)
            #print subset
            #计算子集熵
            p = float(len(subset)) / len(dataSet)
            subset_entropy += p * entropy(subset)
        #计算信息增益
        infoGain = (origin_entropy - subset_entropy)/origin_entropy
        #记录最大IG
        if(infoGain > best_infogain):
            best_feature = i
            best_infogain = infoGain
    return best_feature
 
    
def major_class(classList):
    """
    找出样本数最多的类别
    """
    classCount = {}
    for i in classList:
        if(i not in classCount.keys()):
            classCount[i] = 0
        classCount[i] += 1
    if classCount[1]>classCount[0]:
        result = 1
    else:
        result = 0
    return result
 

def createTree(dataSet,labels):

    labels=labels
    feaLabels=[]
    classList = [example[-1] for example in dataSet]#yes or no

    if(len(classList) == classList.count(classList[0])):#classlist中的第一个元素出现的次数等于classlist的长度 说明全部属于同一个类
        return classList[0]
   
    if(len(dataSet[0]) == 1):#只剩一个特征属性
        majorClass = major_class(classList)
        return majorClass

    best_feature = chooseBestFeature(dataSet)#最优划分特征 的下标号
    best_feaLabel = labels[best_feature]#在labels中找出下标为best_feature的属性
    feaLabels.append(best_feaLabel) #存储最优特征
    del(labels[best_feature])#特征属性中删去最优特征
    feaValue = [example[best_feature] for example in dataSet]
    feaValue = set(feaValue) #获取最优特征的属性值列表 不重复的那种
    tree = {best_feaLabel:{}}#子树的根的key是此次划分的最优特征名，value是再往下递归划分的子树
    for value in feaValue:
        subLabel = labels[:]
        subset = split_data(dataSet,best_feature,value)
        tree[best_feaLabel][value] = createTree(subset,subLabel)
    return tree
 

In [4]:
createTree(dataset,labels)

{' blubbernd': {' normal': ' Tuberkulose ',
  ' fiepend': {'hoch': {'hoch': ' Tuberkulose ',
    'ohne': ' Tuberkulose ',
    'niedrig': ' Lungenentzuendung '}},
  ' blubbernd': ' Lungenentzuendung '}}